In [1]:
import cv2
import numpy as np
import sys
import getopt
import operator

d:\Anaconda3\envs\PyTorch\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
d:\Anaconda3\envs\PyTorch\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
d:\Anaconda3\envs\PyTorch\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
def readImage(filename):
    """
     Read in an image file, errors out if we can't find the file
    :param filename:
    :return: Img object if filename is found
    """
    img = cv2.imread(filename, 0)
    if img is None:
        print('Invalid image:' + filename)
        return None
    else:
        print('Image successfully read...')
        return img

In [3]:
def findCorners(img, window_size, k, thresh):
    """
    Finds and returns list of corners and new image with corners drawn
    :param img: The original image
    :param window_size: The size (side length) of the sliding window
    :param k: Harris corner constant. Usually 0.04 - 0.06
    :param thresh: The threshold above which a corner is counted
    :return:
    """
    d_y, d_x = np.gradient(img)
    Ixx = d_x ** 2
    Ixy = d_y * d_x
    Iyy = d_y ** 2
    h = img.shape[0]
    w = img.shape[1]

    cornerList = []
    newimg = img.copy()
    color_img = cv2.cvtColor(newimg, cv2.COLOR_GRAY2RGB)
    offset = window_size // 2

    for y in range(offset, h - offset):
        for x in range(offset, w - offset):
            # Calculate sum of squares
            windowIxx = Ixx[y - offset:y + offset + 1, x - offset:x + offset + 1]
            windowIxy = Ixy[y - offset:y + offset + 1, x - offset:x + offset + 1]
            windowIyy = Iyy[y - offset:y + offset + 1, x - offset:x + offset + 1]
            Sxx = windowIxx.sum()
            Sxy = windowIxy.sum()
            Syy = windowIyy.sum()

            # Find determinant and trace, use to get corner response
            det = (Sxx * Syy) - (Sxy ** 2)
            trace = Sxx + Syy
            r = det - k * (trace ** 2)

            # If corner response is over threshold, color the point and add to corner list
            if r > thresh:
                #print (x, y, r)
                cornerList.append([x, y, r])
                color_img.itemset((y, x, 0), 100)  #b
                color_img.itemset((y, x, 1), 0)    #g
                color_img.itemset((y, x, 2), 255)  #r
    
    return color_img, cornerList

In [4]:
img = readImage("./img/harris.png")

Image successfully read...


In [5]:
window_size = 5
k = .04 #Corner Response
thresh = 10000 #Corner Response Threshold

In [6]:
if img is not None:
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        if len(img.shape) == 4:
            img = cv2.cvtColor(img, cv2.COLOR_RGBA2GRAY)
        print("Shape: " + str(img.shape))
        print("Size: " + str(img.size))
        print("Type: " + str(img.dtype)) 
        print("Printing Original Image...")
        print(img)
        finalImg, cornerList = findCorners(img, window_size, k, thresh)
        if finalImg is not None:
            cv2.imwrite("finalimage.png", finalImg)

        # Write top 100 corners to file
        cornerList.sort(key=operator.itemgetter(2))
        outfile = open('corners.txt', 'w')
        for i in range(100):
            outfile.write(str(cornerList[i][0]) + ' ' + str(cornerList[i][1]) + ' ' + str(cornerList[i][2]) + '\n')
        outfile.close()

Shape: (384, 384)
Size: 147456
Type: uint8
Printing Original Image...
[[  0   0   0 ... 255 255 255]
 [  0   0   0 ... 255 255 255]
 [  0   0   0 ... 255 255 255]
 ...
 [255 255 255 ...   0   0   0]
 [255 255 255 ...   0   0   0]
 [255 255 255 ...   0   0   0]]
